In [243]:
import numpy as np
import tensorflow as tf
import math
import re
import string
import os

import nltk
import pandas as pd
from langdetect import detect
from wordcloud import WordCloud, STOPWORDS, ImageColorGenerator


In [244]:
new_model = tf.keras.models.load_model('saved_model/my_model.h5')

# Check its architecture
new_model.summary()

Model: "sequential_15"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense_30 (Dense)                │ (None, 512)            │     5,120,512 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_31 (Dense)                │ (None, 5)              │         2,565 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 5,123,079 (19.54 MB)

 Trainable params: 5,123,077 (19.54 MB)

 Non-trainable params: 0 (0.00 B)

 Optimizer params: 2 (12.00 B)

In [245]:
data = pd.read_csv('Datafiniti_Hotel_Reviews.csv', encoding='ISO-8859-1', sep=',')




In [246]:
data = data[[ 'reviews.text', 'reviews.title', 'reviews.rating']]
data['reviews.text'] = data['reviews.title'] + " " + data['reviews.text']

In [247]:
data = data.drop_duplicates()
data = data.dropna()
data.shape

(9995, 3)

In [248]:
import matplotlib.pyplot as plt

def detect_lang(text):
    try:
        return detect(text)
    except:
        return "error"


data['lang'] = data['reviews.text'].apply(lambda x: detect_lang(x))
data = data.loc[data['lang'] == 'en']
data['lang'].value_counts()
data.shape

(9892, 4)

In [249]:
nltk.download('stopwords')
en_stopwords = nltk.corpus.stopwords.words('english')
ps = nltk.PorterStemmer()


def clean_text(text):
    result = "".join([word for word in text if word not in string.punctuation])
    tokens = re.split('\W+', result)
    text = [ps.stem(word) for word in tokens if word not in en_stopwords]
    return text

<>:8: SyntaxWarning: invalid escape sequence '\W'
<>:8: SyntaxWarning: invalid escape sequence '\W'
/var/folders/l1/c9cqs1hx4vx2w36p7pg1dhq80000gn/T/ipykernel_8362/4066190274.py:8: SyntaxWarning: invalid escape sequence '\W'
  tokens = re.split('\W+', result)
[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/pieroneri/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [250]:
import matplotlib.pyplot as plt
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.utils import to_categorical
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
dataRating = data['reviews.rating']
data = data.drop(columns=['reviews.rating'])
dataRating = dataRating.apply(lambda x: x - 5 if x > 5 else x)
dataRating = dataRating.apply(lambda x: math.floor(x))
dataRating

0       5
1       5
2       5
3       2
4       5
       ..
9995    3
9996    4
9997    4
9998    1
9999    1
Name: reviews.rating, Length: 9892, dtype: int64

In [251]:

vectorizer = TfidfVectorizer(max_features=10000, analyzer=clean_text)
X_train, X_val, y_train, y_val = train_test_split(data['reviews.text'], dataRating,test_size=0.2, random_state=23)

X_train_vectorized = vectorizer.fit_transform(X_train)
X_val_vectorized = vectorizer.transform(X_val)


X_train_vectorized = X_train_vectorized.todense()
X_val_vectorized = X_val_vectorized.todense()

In [252]:
encoder = LabelEncoder()
y_train_encoded = to_categorical(encoder.fit_transform(y_train))
y_val_encoded = to_categorical(encoder.transform(y_val))


In [257]:

np.argmax(new_model.predict(X_val_vectorized), axis=1) + 1

62/62 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step


array([1, 5, 5, ..., 5, 5, 5])

In [258]:
dataRating

0       5
1       5
2       5
3       2
4       5
       ..
9995    3
9996    4
9997    4
9998    1
9999    1
Name: reviews.rating, Length: 9892, dtype: int64

In [259]:
#calculate the accuracy
from sklearn.metrics import accuracy_score
accuracy_score(y_val, np.argmax(new_model.predict(X_val_vectorized), axis=1) + 1)

62/62 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step


0.33653360282971195